In [1]:
# dependencies
import pandas as pd
import requests
import json
#import matplotlib.pyplot as plt
from citipy import citipy
import numpy as np #may not need
#import time
import os

# establish openweather API key/url
file_name = "../../../PythonScripts/BootcampExercises/API_Keys/api_keys.json"
data = json.load(open(file_name))
key = data['openweather']
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = url + "appid=" + key + "&units=" + units + "&q="

# get olympic data files
csv_path = os.path.join("../Resources", "GDP_allyears.csv") #do not make changes
gdp = pd.read_csv(csv_path)
csv_path = os.path.join("../Resources", "country_participation.csv") #do not make changes
participation = pd.read_csv(csv_path)
csv_path = os.path.join("../Resources", "country_codes.csv") #do not make changes
codes = pd.read_csv(csv_path)

In [2]:
# join in country codes
participation = participation.merge(codes, on='Country', how='left')

# clean/format participation data set
participation["TotalMedals"] = participation["Gold"] + participation["Silver"] + participation["Bronze"]
participation = participation.fillna(0)
participation["From"] = participation["From"].astype(int)
participation["To"] = participation["To"].astype(int)
participation["Gold"] = participation["Gold"].astype(int)
participation["Silver"] = participation["Silver"].astype(int)
participation["Bronze"] = participation["Bronze"].astype(int)
participation["Sports"] = participation["Sports"].astype(int)
participation["TotalAthletes"] = participation["Participants"].astype(int)
participation = participation.drop('Participants', 1)
participation["TotalMedals"] = participation["TotalMedals"].astype(int)
participation = participation.rename(columns={'Code':'CountryCode','Sports': 'TotalSports','Gold':'TotalGold','Silver':'TotalSilver','Bronze':'TotalBronze'})

In [3]:
# determine number of participation years
gameyears = [1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1994, 1998, 2002, 2006, 2010, 2014]
for index, row in participation.iterrows():
    counter = 0
    To = row[2]
    From = row[1]
    for year in gameyears:
        if year >= From and year <= To:
            counter = counter+1
    participation.loc[index, "TotalYears"] = counter
participation["TotalYears"] = participation["TotalYears"].astype(int)

In [4]:
# find average medals per year per country
participation["AvgMedalsPerYear"] = round(participation["TotalMedals"]/participation["TotalYears"],1)

# find average participants per year per country
participation["AvgAthletePerYear"] = round(participation["TotalAthletes"]/participation["TotalYears"],1)

# find average sports per year per country
participation["AvgSportsPerYear"] = round(participation["TotalSports"]/participation["TotalYears"],1)

# find average participants per sport per country
participation["AvgAthletePerSport"] = round(participation["TotalAthletes"]/participation["TotalSports"],1)

In [5]:
# group GDP data by country, generate total of GDP yearly data points per country
country_groups = gdp.groupby("COUNTRY")
gpd_groups = pd.DataFrame(country_groups["COUNTRY"].value_counts())
gpd_groups = gpd_groups.rename(columns={'COUNTRY': 'TotalRecords'})
gpd_groups = gpd_groups.reset_index(level=0)
gpd_groups = gpd_groups.rename(columns={'COUNTRY': 'CountryCode'})
gpd_groups.index = range(len(gpd_groups))
gpd_groups.head()

# group GDP data by country, generate SUM of all GDP yearly records
country_totals = gdp.groupby("COUNTRY")
gpd_totals = pd.DataFrame(country_totals["USDperCapita"].sum())
gpd_totals = gpd_totals.rename(columns={'COUNTRY': 'TotalGDP'})
gpd_totals = gpd_totals.reset_index(level=0)
gpd_totals = gpd_totals.rename(columns={'COUNTRY': 'CountryCode'})
gpd_totals.index = range(len(gpd_groups))
gpd_totals.head()

# join in country codes, calculate average GDP
GDP = gpd_groups.merge(gpd_totals, on='CountryCode', how='left')
GDP["AvgGDP"] = round(GDP["USDperCapita"]/GDP["TotalRecords"],2)

# join in country codes, calculate average GDP, produce final table for GDP regression
GDP = GDP.merge(participation, on='CountryCode', how='left')
GDP = GDP[pd.notnull(GDP['Country'])]

In [8]:
#generate random city name, check for duplicates, check for response; append data to list until there are 500 records

cities = []
city_data = []

while len(city_data) < 5:
    randcity = citipy.nearest_city((np.random.randint(low=-9000, high=9000)*.01), (np.random.randint(low=-18000, high=18000)*.01))
    city = randcity.city_name
    if city not in cities:
            target_url = query_url + city.replace(" ","+")
            response = requests.get(target_url).json()
            cities.append(city)
            if response["cod"] == 200:
                city_data.append(response)
                print("("+str(len(city_data))+")  "+response.get("name")+":  "+target_url)
                
#extract fields to dataframe, save to .csv

header = ['City']
weatherdata = pd.DataFrame([data.get("name") for data in city_data], columns=header)
weatherdata['Latitude'] = pd.DataFrame([data.get("coord").get("lat") for data in city_data])
weatherdata['Temperature'] = pd.DataFrame([data.get("main").get("temp") for data in city_data])
weatherdata.to_csv("weatherdata.csv", encoding='utf-8', index=False)

(1)  Rikitea:  http://api.openweathermap.org/data/2.5/weather?appid=40d439259718ef3f486d5ccb188390f5&units=imperial&q=rikitea
(2)  Atuona:  http://api.openweathermap.org/data/2.5/weather?appid=40d439259718ef3f486d5ccb188390f5&units=imperial&q=atuona
(3)  Mataura:  http://api.openweathermap.org/data/2.5/weather?appid=40d439259718ef3f486d5ccb188390f5&units=imperial&q=mataura
(4)  Torbay:  http://api.openweathermap.org/data/2.5/weather?appid=40d439259718ef3f486d5ccb188390f5&units=imperial&q=torbay
(5)  Hobart:  http://api.openweathermap.org/data/2.5/weather?appid=40d439259718ef3f486d5ccb188390f5&units=imperial&q=hobart


In [7]:
participation.head()

,Country,From,To,TotalSports,TotalGold,TotalSilver,TotalBronze,CountryCode,TotalMedals,TotalAthletes,TotalYears,AvgMedalsPerYear,AvgAthletePerYear,AvgSportsPerYear,AvgAthletePerSport
0,Albania,2006,2014,1,0,0,0,ALB,0,2,3,0.0,0.7,0.3,2.0
1,Algeria,1992,2010,2,0,0,0,ALG,0,7,6,0.0,1.2,0.3,3.5
2,American Samoa,1994,1994,1,0,0,0,ASA,0,2,1,0.0,2.0,1.0,2.0
3,Andorra,1976,2014,4,0,0,0,AND,0,28,11,0.0,2.5,0.4,7.0
4,Argentina,1928,2014,8,0,0,0,ARG,0,136,21,0.0,6.5,0.4,17.0
